https://github.com/IgnacioPardo/Tecnologias_Exponenciales_2023/blob/main/Consigna_NN.md
https://github.com/IgnacioPardo/SNN/tree/main

# Trabajo Práctico Redes Neuronales

## Introducción

El objetivo de este trabajo es plantear una red neuronal con una capa de entrada con $n$ neuronas y una capa de salida con $m$ salidas para resolver un problema de su propía creación.

Se evaluara la correcta implementación de la red neuronal y la explicación de los elementos de la teoría vistos en clase.

## Consigna

A partir del siguiente Notebook visto en clase [SNN](https://github.com/IgnacioPardo/SNN/blob/main/SNN.ipynb), se pide:

1. Plantear un problema a resolver con una red neuronal de una capa de entrada y una capa de salida. Las salidas no deben ser excluyentes, es decir, pueden pertenecer a mas de una clase. Algunos ejemplos son: A partir de caracteristicas de una vivienda, determinar si la vivienda es de algun estilo (moderna, agradable, tradicional). Pueden generar ustedes un pequeño dataset para utilizar o buscar en Kaggle.
2. Modificar la estructura de la red neuronal para que admita $n$ neuronas de entrada y $m$ neuronas de salida.

Ademas deben indicar los elementos de la teoría que vimos en clase que identifiquen en el código y explicarlos (Estructura de la Red, Función de activación, Forward propagation, Back propagation y Gradiente descendente).

## Bonus

A. ¿Como modificarían la estructura de la red para que admita $k$ capas ocultas con $n$ neuronas cada una? Describir como sería el algoritmo de forward propagation en este caso, no es necesario implementar el algoritmo de back propagation.

B. Investigar la función de activación Softmax y explicar como se puede utilizar para resolver un problema de clasificación con $m$ clases mutualmente excluyentes. ¿Que habría que modificar en el código para utilizar esta función de activación?

## Entrega

La entrega consiste en un notebook de Jupyter con el código y las explicaciones correspondientes. La fecha de entrega es el 29/10/2023 a través del siguiente [Google Forms]().

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import tabulate
import networkx as nx
from random import random as rand
import itertools

In [3]:
def sigmoid(x):
    return 1 / ( 1 + np.exp(-x) )

# sigmoid = lambda x : 1 / ( 1 + np.exp(-x) )

def sigmoid_derivative(x):
    return x * (1 - x)

In [4]:
df = pd.read_csv("seattle-weather.csv")
weather_tof = pd.get_dummies(df['weather'])
df = pd.concat([df, weather_tof], axis=1)
df = df.drop(columns=['date', 'weather'])
df

,precipitation,temp_max,temp_min,wind,drizzle,fog,rain,snow,sun
0,0.0,12.8,5.0,4.7,True,False,False,False,False
1,10.9,10.6,2.8,4.5,False,False,True,False,False
2,0.8,11.7,7.2,2.3,False,False,True,False,False
3,20.3,12.2,5.6,4.7,False,False,True,False,False
4,1.3,8.9,2.8,6.1,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...
1456,8.6,4.4,1.7,2.9,False,False,True,False,False
1457,1.5,5.0,1.7,1.3,False,False,True,False,False
1458,0.0,7.2,0.6,2.6,False,True,False,False,False
1459,0.0,5.6,-1.0,3.4,False,False,False,False,True


In [5]:
df_train = df.iloc[0:5].to_numpy()
df_test = df.iloc[5:].to_numpy()
df_test

array([[2.5, 4.4, 2.2, ..., True, False, False],
       [0.0, 7.2, 2.8, ..., True, False, False],
       [0.0, 10.0, 2.8, ..., False, False, True],
       ...,
       [0.0, 7.2, 0.6, ..., False, False, False],
       [0.0, 5.6, -1.0, ..., False, False, True],
       [0.0, 5.6, -2.1, ..., False, False, True]], dtype=object)

In [6]:
#usar las columnas de wind	drizzle	fog	rain	snow	sun como salida 
training_outputs = df[['wind', 'drizzle', 'fog', 'rain', 'snow', 'sun']].to_numpy()
training_inputs = df[['precipitation',	'temp_max',	'temp_min']].to_numpy()
training_inputs
training_outputs



array([[4.7, True, False, False, False, False],
       [4.5, False, False, True, False, False],
       [2.3, False, False, True, False, False],
       ...,
       [2.6, False, True, False, False, False],
       [3.4, False, False, False, False, True],
       [3.5, False, False, False, False, True]], dtype=object)

In [23]:
#hacer los sinaptic weights
n = training_inputs.shape[1]
synaptic_weights = 2 + np.random.random((3, 1)) - 1
w = synaptic_weights
x = training_inputs
eval_NN = lambda x, w: sigmoid(np.dot(x, w))

w = w.astype('float64')



In [22]:
for iteration in range(50000):

    # Arranca con los valores de training
    input_layer = x

    # Genera un output haciendo la suma de los valores de input multiplicados por su peso sinaptico, y aplicandole la función sigmoide para normalizar a -1, 1
    outputs = eval_NN(input_layer, w)

    # Calculo el error
    error = training_outputs - outputs

    # Ajusto los pesos en funcion al error y al resultado obtenido
    adjustments = error * sigmoid_derivative(outputs)
    w += np.dot(input_layer.T, adjustments)

UFuncTypeError: Cannot cast ufunc 'add' output from dtype('O') to dtype('float64') with casting rule 'same_kind'